## Import library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

## Read data set

In [5]:
path = 'file/'
#path = '../input/'
train = pd.read_csv(path + 'train.csv',parse_dates=['date'],usecols=[1,2,3,4,5],dtype= {'onpromotion':str},
                   converters= {'unit_sales': lambda u: float(u) if float(u)>0 else 0},skiprows=range(1, 124035460))
print("Number of rows and columns in the train data set:",train.shape)


Number of rows and columns in the train data set: (1461581, 5)


In [3]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [ ]:
# Log tranform
train['unit_sales'] = train['unit_sales'].apply(np.log1p)

In [ ]:
#fill gap in data
u_dates = train['date'].unique()
u_stores = train['store_nbr'].unique()
u_items = train['item_nbr'].unique()
train.set_index(['date','store_nbr','item_nbr'],inplace=True)
train = train.reindex(pd.MultiIndex.from_product([u_dates,u_stores,u_items],
                     names=['date','store_nbr','item_nbr']))


In [ ]:
# Fill na
train.loc[:,'unit_sales'].fillna(0,inplace=True)
train.loc[:,'onpromotion'].fillna('False',inplace=True)


## Calculate mean

In [ ]:
train = train.groupby(['item_nbr','store_nbr','onpromotion'])['unit_sales'].mean().to_frame('unit_sales')

In [ ]:
# Inverse Transform
train['unit_sales'] = train['unit_sales'].apply(np.expm1)


## Submission

In [ ]:
pd.read_csv(path + 'test.csv', usecols=[0,2,3,4], dtype = {'onpromotion': str}).set_index(
['item_nbr','store_nbr','onpromotion']).join(
train,how = 'left').fillna(0).to_csv('mean.csv',index=None)
